In [1]:
#!pip install opencv-python

In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CLASSIFICATION USING SEGMENTATION MODEL OUTPUT BCDR MASS LESION
import numpy as np
import tensorflow.keras
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
#%matplotlib inline
keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
#import pandas as pd
import os
#import matplotlib.pyplot as plt
#from tensorflow.python.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras import Sequential, Model

vid='v002_3' #last number is the cross-validation fold number

In [2]:
from tensorflow import keras
model = keras.models.load_model("mass_class_VGG16_v002_"+vid[-1]+".h5")

In [3]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
len(model.layers)

26

In [5]:
# Image preprocessing and data augmentation
import cv2
IMG_SIZE=224
batch_size=8

train_datagen = ImageDataGenerator(
                         horizontal_flip = True,
                         vertical_flip = True,
                         rotation_range = 90,
                         fill_mode="constant",
                         width_shift_range = 0.2,
                         zoom_range = 0.2,
                         rescale=1./255,
                         height_shift_range = 0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train0'+vid[-1], 
                                                  target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('valid0'+vid[-1], 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=True)


Found 182 images belonging to 2 classes.
Found 46 images belonging to 2 classes.


In [6]:
# Predicted object delete threshold = 30
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('test_upp_xcept', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='binary',
                                                 shuffle=False)


# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict(test_generator)
y_pred=(Y_pred>0.5).astype(int)
ras=roc_auc_score(test_generator.classes, Y_pred)
prec=precision_score(test_generator.classes, y_pred,pos_label=1)
rec=recall_score(test_generator.classes, y_pred,pos_label=1)
f1s=f1_score(test_generator.classes, y_pred,pos_label=1)
mcc=matthews_corrcoef(test_generator.classes, y_pred)

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))
print('Accuracy:', accuracy_score(test_generator.classes, y_pred))
print('ROC AUC score:', ras)
print('Precision score:', prec)
print('Recall score:', rec)
print('F1 score:', f1s)
print('MCC score:', mcc)

Found 42 images belonging to 2 classes.
6/6 [==============================] - 36s 264ms/step
Confusion Matrix:
[[23  2]
 [ 8  9]]
Accuracy: 0.7619047619047619
ROC AUC score: 0.8188235294117647
Precision score: 0.8181818181818182
Recall score: 0.5294117647058824
F1 score: 0.6428571428571428
MCC score: 0.5017203455689636
